In [1]:
from __future__ import print_function, division
import matplotlib.pyplot as plt
import pandas as pd

from neuralnilm.data.loadactivations import load_nilmtk_activations
from neuralnilm.data.syntheticaggregatesource import SyntheticAggregateSource
from neuralnilm.data.datapipeline import DataPipeline
from neuralnilm.data.processing import DivideBy, IndependentlyCenter
from neuralnilm.data.datathread import DataThread

In [2]:
NILMTK_FILENAME = '/data/mine/vadeec/merged/ukdale.h5'
TARGET_APPLIANCE = 'kettle'
SEQ_LENGTH = 256
SAMPLE_PERIOD = 6
STRIDE = SEQ_LENGTH
WINDOWS = {
    'train': {
        1: ("2014-01-01", "2014-02-01")
    },
    'unseen_activations_of_seen_appliances': {
        1: ("2014-02-02", "2014-02-08")                
    },
    'unseen_appliances': {
        2: ("2013-06-01", "2013-06-07")
    }
}

LOADER_CONFIG = {
    'nilmtk_activations': dict(
        appliances=['kettle', 'microwave', 'washing machine'],
        filename=NILMTK_FILENAME,
        sample_period=SAMPLE_PERIOD,
        windows=WINDOWS
    )
}

In [3]:
from neuralnilm.data.stridesource import StrideSource

In [4]:
stride_source = StrideSource(
    target_appliance=TARGET_APPLIANCE,
    seq_length=SEQ_LENGTH,
    filename=NILMTK_FILENAME,
    windows=WINDOWS,
    sample_period=SAMPLE_PERIOD,
    stride=STRIDE
)

In [5]:
nilmtk_activations = load_nilmtk_activations(**LOADER_CONFIG['nilmtk_activations'])

In [6]:
from neuralnilm.data.realaggregatesource import RealAggregateSource

In [7]:
ras = RealAggregateSource(
    activations=nilmtk_activations,
    target_appliance=TARGET_APPLIANCE,
    seq_length=SEQ_LENGTH,
    filename=NILMTK_FILENAME,
    windows=WINDOWS,
    sample_period=SAMPLE_PERIOD
)

In [8]:
%prun ras.get_sequence().next()

In [8]:
source = SyntheticAggregateSource(
    activations=nilmtk_activations,
    target_appliance=TARGET_APPLIANCE,
    seq_length=SEQ_LENGTH,
    allow_incomplete_target=False,
    sample_period=SAMPLE_PERIOD
)

In [9]:
sample = source.get_batch(num_seq_per_batch=1024).next()

In [10]:
pipeline = DataPipeline(
    sources=[source, ras, stride_source],
    num_seq_per_batch=64,
    input_processing=[DivideBy(sample.before_processing.input.flatten().std()), IndependentlyCenter()],
    target_processing=[DivideBy(sample.before_processing.target.flatten().std())]
)

# Disaggregation

nilmtk_disag_source = NILMTKDisagSource(
    filename=NILMTK_FILENAME,
    target_appliance=TARGET_APPLIANCE,
    seq_length=SEQ_LENGTH,
    buildings=[5],
    window_per_building={},
    stride=STRIDE,
    sample_period=SAMPLE_PERIOD
)

disag_pipeline = deepcopy(pipeline)
disag_pipeline.source = nilmtk_disag_source

disaggregator = Disaggregator(
    pipeline=disag_pipeline,
    output_path=PATH  # 
)

Disagregator ideas:

* make a copy of pipeline but swap source for a NILMTKDisagSource
* NILMTKDisagSource loads all data into memory (?) and iterates over chunks of it (get seq_length from pipeline.source.seq_length)
* 

In [11]:
from lasagne.layers import InputLayer, RecurrentLayer, DenseLayer, ReshapeLayer

def get_net_0(input_shape, target_shape=None):
    NUM_UNITS = {
        'dense_layer_0': 100,
        'dense_layer_1':  50,
        'dense_layer_2': 100
    }

    if target_shape is None:
        target_shape = input_shape
    
    # Define layers
    input_layer = InputLayer(
        shape=input_shape
    )
    
    # Dense layers
    dense_layer_0 = DenseLayer(
        input_layer, 
        num_units=NUM_UNITS['dense_layer_0']
    )
    dense_layer_1 = DenseLayer(
        dense_layer_0, 
        num_units=NUM_UNITS['dense_layer_1']
    )
    dense_layer_2 = DenseLayer(
        dense_layer_1, 
        num_units=NUM_UNITS['dense_layer_2']
    )
    
    # Output
    final_dense_layer = DenseLayer(
        dense_layer_2,
        num_units=target_shape[1] * target_shape[2],
        nonlinearity=None
    )
    output_layer = ReshapeLayer(
        final_dense_layer,
        shape=target_shape
    )
    return output_layer

In [12]:
from neuralnilm.net import Net

batch = pipeline.get_batch()
output_layer = get_net_0(
    batch.after_processing.input.shape, 
    batch.after_processing.target.shape
)
net = Net(output_layer, tags=['AE'], description="Just testing new NeuralNILM code!")

In [13]:
from neuralnilm.trainer import Trainer
from neuralnilm.metrics import Metrics

trainer = Trainer(
    net=net,
    data_pipeline=pipeline,
    experiment_id=["5"],
    metrics=Metrics(state_boundaries=[4]),
    learning_rates={0: 1E-2, 10000: 1E-3},
    repeat_callbacks=[
        (5000, Trainer.save_params),
        (5000, Trainer.plot_estimates),
        ( 500, Trainer.validate)
    ]
)

Database already has an experiment with _id == 5. Should the old experiment be deleted (both from the database and from disk)? Or quit? [Q/d] d


INFO:neuralnilm.trainer:Deleting documents for old experiment.
INFO:neuralnilm.trainer:Directory exists = '/home/dk3810/temp/neural_nilm/output/5'
INFO:neuralnilm.trainer:  Deleting directory.


In [ ]:
contents = [(['data'], {'activations': LOADER_CONFIG})]
trainer.submit_report(additional_report_contents=contents)

{'_id': '5',
 'data': {'activations': {'nilmtk_activations': {'appliances': ['kettle',
     'microwave',
     'washing machine'],
    'filename': '/data/mine/vadeec/merged/ukdale.h5',
    'sample_period': 6,
    'windows': {'train': {'1': ('2014-01-01', '2014-02-01')},
     'unseen_activations_of_seen_appliances': {'1': ('2014-02-02',
       '2014-02-08')},
     'unseen_appliances': {'2': ('2013-06-01', '2013-06-07')}}}},
  'pipeline': {'input_processing': [{'divisor': 621.045654296875,
     'name': 'DivideBy'},
    {'name': 'IndependentlyCenter'}],
   'num_seq_per_batch': 64,
   'rng_seed': None,
   'source_probabilities': [0.3333333333333333,
    0.3333333333333333,
    0.3333333333333333],
   'sources': {'0': {'allow_incomplete_distractors': True,
     'allow_incomplete_target': False,
     'distractor_inclusion_prob': 0.25,
     'include_incomplete_target_in_output': True,
     'name': 'SyntheticAggregateSource',
     'num_batches_for_validation': 16,
     'rng_seed': None,
     's

In [ ]:
trainer.fit(500000)

INFO:neuralnilm.trainer:Starting training for 500000 iterations.
INFO:neuralnilm.trainer:Iteration 0: Change learning rate to 1.0E-02
INFO:neuralnilm.trainer:Compiling train cost function...
INFO:neuralnilm.trainer:Done compiling cost function.


   Update # |  Train cost  | Secs per update | Source ID
------------|--------------|-----------------|-----------
          0 |   0.189608  |    2.470165     |   2

INFO:neuralnilm.trainer:Iteration 0: Saving params.
INFO:neuralnilm.trainer:Done saving params.
INFO:neuralnilm.trainer:Iteration 0: Plotting estimates.
INFO:neuralnilm.net:Compiling deterministic output function...
INFO:neuralnilm.net:Done compiling deterministic output function.
INFO:neuralnilm.trainer:Done plotting.



          1 |   0.165158  |    0.081116     |   2
          2 |   1.137131  |    0.036718     |   0
          3 |   0.993763  |    0.049879     |   0
          4 |   0.918286  |    0.151728     |   1
          5 |   1.164499  |    0.148896     |   1
          6 |   0.770006  |    0.024396     |   0
          7 |   0.186775  |    0.043550     |   2
          8 |   0.242785  |    0.050035     |   2
          9 |   0.789430  |    0.044947     |   0
         10 |   0.232419  |    0.045485     |   2
         11 |   1.252085  |    0.080370     |   0
         12 |   0.877916  |    0.042871     |   0
         13 |   0.106668  |    0.045997     |   2
         14 |   1.141130  |    0.073120     |   0
         15 |   0.236916  |    0.049077     |   2
         16 |   0.978012  |    0.042017     |   0
         17 |   1.090046  |    0.051714     |   0
         18 |   0.777272  |    0.047713     |   0
         19 |   0.883186  |    0.026989     |   0
         20 |   0.183250  |    0.081775     |   2

/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:958: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:958: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
INFO:neuralnilm.trainer:Iteration 83: Finished training epoch


...


        795 |   0.623937  |    0.081733     |   0
        796 |   0.442293  |    0.052827     |   0
        797 |   0.614707  |    0.198732     |   1
        798 |   0.677347  |    0.143430     |   1
        799 |   0.570984  |    0.075734     |   0
        800 |   0.534331  |    0.023561     |   0
        801 |   0.445073  |    0.075908     |   0
        802 |   0.593836  |    0.046669     |   0
        803 |   0.170032  |    0.041951     |   2
        804 |   0.595138  |    0.072955     |   0
        805 |   0.155199  |    0.005766     |   2
        806 |   0.592017  |    0.079410     |   0
        807 |   0.178773  |    0.029846     |   2
        808 |   0.189794  |    0.063525     |   2
        809 |   0.497116  |    0.199044     |   1
        810 |   0.097207  |    0.008049     |   2
        811 |   0.180514  |    0.086221     |   2
        812 |   0.599732  |    0.153919     |   1
        813 |   0.140706  |    0.034333     |   2
        814 |   0.641338  |    0.188920     |   1

INFO:neuralnilm.trainer:Iteration 864: Finished training epoch



        864 |   0.518287  |    0.017321     |   0
        865 |   0.524559  |    0.188201     |   1
        866 |   0.400800  |    0.024461     |   0
        867 |   0.171717  |    0.083025     |   2
        868 |   0.473514  |    0.021041     |   0
        869 |   0.150604  |    0.074672     |   2
        870 |   0.170683  |    0.042500     |   2
        871 |   0.568483  |    0.152712     |   1
        872 |   0.451517  |    0.044629     |   0
        873 |   0.360861  |    0.153084     |   1
        874 |   0.600209  |    0.165498     |   1
        875 |   0.566035  |    0.134667     |   1
        876 |   0.186700  |    0.017423     |   2
        877 |   0.094730  |    0.077886     |   2
        878 |   0.174979  |    0.030444     |   2
        879 |   0.139576  |    0.045536     |   2
        880 |   0.468820  |    0.042941     |   0
        881 |   0.156455  |    0.082456     |   2
        882 |   0.150791  |    0.006709     |   2
        883 |   0.052378  |    0.093267     |   2

INFO:neuralnilm.trainer:Iteration 933: Finished training epoch



        933 |   0.588465  |    0.138816     |   1
        934 |   0.169647  |    0.046863     |   2
        935 |   0.386906  |    0.200434     |   1
        936 |   0.484577  |    0.018116     |   0
        937 |   0.594704  |    0.078179     |   0
        938 |   0.143933  |    0.030458     |   2
        939 |   0.492140  |    0.221183     |   1
        940 |   0.517774  |    0.160732     |   1
        941 |   0.592972  |    0.144553     |   1
        942 |   0.163561  |    0.008222     |   2
        943 |   0.426736  |    0.083596     |   0
        944 |   0.182297  |    0.028246     |   2
        945 |   0.582861  |    0.194457     |   1
        946 |   0.090046  |    0.008842     |   2
        947 |   0.471592  |    0.076828     |   0
        948 |   0.413632  |    0.034748     |   0
        949 |   0.164508  |    0.072909     |   2
        950 |   0.134608  |    0.017020     |   2
        951 |   0.147267  |    0.088963     |   2
        952 |   0.536994  |    0.157191     |   1